In [64]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import transformers
from torch.optim import Adam
from tqdm import tqdm

In [65]:
import torch
from torch.utils.data import Dataset, DataLoader, Subset
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration, T5TokenizerFast

In [66]:
from utils import FinQA_Dataset

### Declarations

In [67]:
BATCH_SIZE = 4
NUM_EPOCHS = 2
TESTING = True

In [68]:
TOKENIZER = T5Tokenizer.from_pretrained("t5-base")
MODEL = T5ForConditionalGeneration.from_pretrained("t5-base", return_dict=True)
OPTIMIZER = Adam(MODEL.parameters(), lr=0.0001)


/Users/ashish1610dhiman/opt/anaconda3/envs/dlt_project/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:199: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


### Read Data and create DataLoader

In [69]:
finqa_train = torch.load('../finqa_dataset/finqa_train.pth')
finqa_valid = torch.load('../finqa_dataset/finqa_valid.pth')
# torch.save(finqa_test, '../finqa_dataset/finqa_test.pth')

In [70]:
if TESTING:
    train_loader = DataLoader(Subset(finqa_train, range(100)), batch_size=BATCH_SIZE)
    val_loader = DataLoader(Subset(finqa_valid, range(10)), batch_size=BATCH_SIZE)
else:
    train_loader = DataLoader(finqa_train, batch_size=BATCH_SIZE)
    val_loader = DataLoader(finqa_valid, batch_size=BATCH_SIZE)

### Train Loop

In [71]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cpu')

In [72]:
train_loss = 0
val_loss = 0
b=0
train_hist=[]
val_hist = []
num_batches = len(train_loader)
print_every=num_batches//10

for epoch in range(NUM_EPOCHS):
    MODEL.train()
    for batch in tqdm(train_loader, desc="Training batches"):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        decoder_attention_mask = batch["decoder_attention_mask"].to(DEVICE)
        outputs = MODEL(input_ids=input_ids,attention_mask=attention_mask,
                          labels=labels,decoder_attention_mask=decoder_attention_mask)
        OPTIMIZER.zero_grad()
        outputs.loss.backward()
        OPTIMIZER.step()
        train_loss += outputs.loss.item()
        b+=1
        if b%(print_every)==0:
            print(f"Batch {b+1}/{num_batches} -> Train loss: {train_loss/b:.4f}")
    train_hist.append(train_loss / len(train_loader))
    #Evaluation
    MODEL.eval()
    for batch in tqdm(val_loader, desc="Validation batches"):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        decoder_attention_mask = batch["decoder_attention_mask"].to(DEVICE)
        with torch.no_grad():
            outputs = MODEL(input_ids=input_ids, attention_mask=attention_mask,
                          labels=labels, decoder_attention_mask=decoder_attention_mask)
        val_loss += outputs.loss.item()
    val_hist.append(val_loss / len(val_loader))
    print(f"Epoch {epoch+1}/{NUM_EPOCHS} -> Train loss: {train_hist[-1]:.4f}\tValidation loss: {val_hist[-1]:.4f}")

Training batches:   0%|          | 0/25 [00:00<?, ?it/s]

Training batches:   8%|▊         | 2/25 [01:16<14:42, 38.36s/it]

Batch 3/25 -> Train loss: 6.2316


Training batches:  16%|█▌        | 4/25 [02:33<13:25, 38.37s/it]

Batch 5/25 -> Train loss: 4.2467


...

Validation batches: 100%|██████████| 3/3 [00:22<00:00,  7.41s/it]


Epoch 1/2 -> Train loss: 1.6383	Validation loss: 0.4996


Training batches:   8%|▊         | 2/25 [01:31<17:29, 45.61s/it]

Batch 3/25 -> Train loss: 21.3276


Training batches:  12%|█▏        | 3/25 [02:20<17:12, 46.95s/it]

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2,)
axes[0].plot(train_hist)
axes[0].set_title('Training Loss')

axes[1].plot(val_hist)
axes[1].set_title('Validation Loss')

In [ ]:
MODEL.save_pretrained("./finqa_finetune_t5.pth")